# "Kaggle Titanic Dataset Top 1% Solution"
> "Top1%: FastAI Beginner Tutorial"

- toc: true
- badges: true
- comments: true
- categories: [fastai, kaggle, datascience]
- author: Hitesh Kumar
- image: images/copied_from_nb/img/ship.png

![Cover Image](img/ship.png)

# What are we doing?

In this blog, we use kaggle titanic dataset. Which got data of all the people who were in titanic fiasco accident. Sp, we are using some part of data to train model so that it can predict if someone survived in titanic fiasco. 

# Intro:

In this blog, i am explaining about how you can get kaggle top 1% score at beginner [dataset](https://www.kaggle.com/c/titanic). But for the better result. I am using [this data](https://www.kaggle.com/pavlofesenko/titanic-extended). Which is a bit extension of kaggle original one.

We are not USING EDA here. This post is to show you how can a powerful tool like fastai can make life so simple. Only automatic preprocessing data in fastai and training model and getting score of top1%.


Here, we are using dataset which is a bit extension of classic titanic dataset. you can check out this: https://www.kaggle.com/pavlofesenko/titanic-extended

And It is fair to use this, i reckon. We are not altering test data at all. Only using new features(Like how we generate new features). Number of rows remains same in both test and training data.

We will be using newly released fastai2.0 library and standard sklearn. fastai2.0 is worth checking out https://docs.fast.ai/ It is more fast and efficient. With this, we can minimize our effort for preprocessing and gain better results.


In [ ]:
#hide

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic-extended/train.csv
/kaggle/input/titanic-extended/test.csv
/kaggle/input/titanic-extended/full.csv


Installing fastai2, yes its not preinstalled. We would need to install on our own. Incase you are running this notebook on kaggle workspace.

 > Warning: If you get some sort of error in importing fastai modules. I would recommend changing notebook environment to `Always use latest environment Always get the latest package versions, but you may have to modify your code`



In [ ]:
#hide_output
!pip install fastai2

     |████████████████████████████████| 179 kB 603 kB/s 
     |████████████████████████████████| 748.8 MB 11 kB/s 
     |████████████████████████████████| 5.9 MB 17.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.5.1
    Uninstalling torch-1.5.1:
      Successfully uninstalled torch-1.5.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.6.0a0+35d732a
    Uninstalling torchvision-0.6.0a0+35d732a:
      Successfully uninstalled torchvision-0.6.0a0+35d732a
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

kornia 0.3.2 requires torch<1.6.0,>=1.5.0, but you'll have torch 1.6.0 which is incompatible.
allennlp 1.0.0 requires torch<1.6.0,>=1.5.0, but you'll have torch 1.6.0 which is incomp

In [ ]:
#hide_output
!pip install fastcore==0.1.35 ##Currently supported with fastai2

  Attempting uninstall: fastcore
    Found existing installation: fastcore 1.0.9
    Uninstalling fastcore-1.0.9:
      Successfully uninstalled fastcore-1.0.9
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
import fastcore
fastcore.__version__

'0.1.35'

In [ ]:
import fastai2
fastai2.__version__

'0.0.30'

#  Importing and Loading files

Now, starting with importing the intended modules. 

In [ ]:
from fastai2.tabular.all import *

In [ ]:
#collapse-hide
df_test= pd.read_csv('/kaggle/input/titanic-extended/test.csv')
df_train= pd.read_csv('../input/titanic-extended/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,WikiId,Name_wiki,Age_wiki,Hometown,Boarded,Destination,Lifeboat,Body,Class
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,S,691.0,"Braund, Mr. Owen Harris",22.0,"Bridgerule, Devon, England",Southampton,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,...,C,90.0,"Cumings, Mrs. Florence Briggs (née Thayer)",35.0,"New York, New York, US",Cherbourg,"New York, New York, US",4,NaN,1.0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,S,865.0,"Heikkinen, Miss Laina",26.0,"Jyväskylä, Finland",Southampton,New York City,14?,NaN,3.0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,S,127.0,"Futrelle, Mrs. Lily May (née Peel)",35.0,"Scituate, Massachusetts, US",Southampton,"Scituate, Massachusetts, US",D,NaN,1.0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,S,627.0,"Allen, Mr. William Henry",35.0,"Birmingham, West Midlands, England",Southampton,New York City,NaN,NaN,3.0


Just taking abit statistical look into data.

In [ ]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,WikiId,Age_wiki,Class
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,889.000000,887.000000,889.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,665.466817,29.322063,2.307087
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,380.796997,13.930089,0.837713
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,1.000000,0.420000,1.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,336.000000,20.000000,2.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,672.000000,28.000000,3.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,996.000000,38.000000,3.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1314.000000,74.000000,3.000000


Before proceeding with anything else. It's important to check, if our data contains any missing values or not. For Calculating the average null values we have in our data. It is important to know. Just to get intuition about data 

In [ ]:
#collapse-hide
df_train.isnull().sum().sort_index()/len(df_train)

Age            0.198653
Age_wiki       0.004489
Boarded        0.002245
Body           0.902357
Cabin          0.771044
Class          0.002245
Destination    0.002245
Embarked       0.002245
Fare           0.000000
Hometown       0.002245
Lifeboat       0.612795
Name           0.000000
Name_wiki      0.002245
Parch          0.000000
PassengerId    0.000000
Pclass         0.000000
Sex            0.000000
SibSp          0.000000
Survived       0.000000
Ticket         0.000000
WikiId         0.002245
dtype: float64

Getting names of columns according to its datatype.

This will allow us to understand which feature is int, float, and object(categorical)


In [ ]:
#collapse-hide
df_train.dtypes
g_train =df_train.columns.to_series().groupby(df_train.dtypes).groups
g_train

{int64: ['PassengerId', 'Pclass', 'SibSp', 'Parch'], float64: ['Survived', 'Age', 'Fare', 'WikiId', 'Age_wiki', 'Class'], object: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Name_wiki', 'Hometown', 'Boarded', 'Destination', 'Lifeboat', 'Body']}

Now this might be a bit puzzling if you are new to fastai. And its completely alright.

cat_names refers to the features which are categorical.

cont_names refers to the features which are continuous. For example : int and float

fastai needs them in order to do preprocessing for you properly


In [ ]:
cat_names= [
        'Name', 'Sex', 'Ticket', 'Cabin', 
        'Embarked', 'Name_wiki', 'Hometown', 
        'Boarded', 'Destination', 'Lifeboat', 
        'Body'
]

cont_names = [ 
    'PassengerId', 'Pclass', 'SibSp', 'Parch', 
    'Age', 'Fare', 'WikiId', 'Age_wiki','Class'
 ]



# FastAI2

now we are diving into fastai2. looking down at code. Might be new and intimidating. But no worries,
following the documentation helped me. https://docs.fast.ai/tutorial.tabular

 

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df_train))

to = TabularPandas(df_train, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Survived',
                   splits=splits)

Here we are using fastai TabularPandas library. Which will do all the preprocessing for us.
before that splitting our data into validation set to have a fair amount of idea that we are not overfitting the data. valid_pct= 0.2 means (as you may have guessed by now) it means 20% validation data.

y_names, using column we are `target to be predicted`.
procs are [Categorify, FillMissing,Normalize], Convering objects into category, filling the missing value and normalizing the content for quick processing.


 > Note: Tabularpandas function creates a new column if there is any missing value spotted in any particular feature(only float or int column). for example: lets say Fare has a missing value somewhere. It will create a new column named 'Fare_na' where it will have integer values (1 or 2).

In [ ]:
#collapse-hide

g_train =to.train.xs.columns.to_series().groupby(to.train.xs.dtypes).groups
g_train

{int8: ['Sex', 'Embarked', 'Boarded', 'Lifeboat', 'Body', 'Age_na', 'WikiId_na', 'Age_wiki_na', 'Class_na'], int16: ['Name', 'Ticket', 'Cabin', 'Name_wiki', 'Hometown', 'Destination'], float64: ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare', 'WikiId', 'Age_wiki', 'Class']}

Here you can see 'Age_na', 'WikiId_na', 'Age_wiki_na', 'Class_na' . Which were created becoz their columns had missing values

Generally what happens is, we cannot use the output of tabularpandas straightaway. So we need to access it as follows.. just taking peek at train data(for validation data replace train with valid)

In [ ]:
#hide
to.train.xs.Age_na.head()

430    1
158    2
453    1
750    1
250    2
Name: Age_na, dtype: int8

In [ ]:
#collapse-show
to.train

     PassengerId  Survived    Pclass  Name  Sex       Age     SibSp     Parch  \
430    -0.059897       1.0 -1.585850    93    2 -0.120416 -0.472176 -0.462981   
158    -1.123753       0.0  0.827621   768    2 -0.120416 -0.472176 -0.462981   
453     0.030061       1.0 -1.585850   291    2  1.492635  0.468220 -0.462981   
750     1.191698       1.0 -0.379115   853    1 -1.963903  0.468220  0.811558   
250    -0.763919       0.0  0.827621   674    2 -0.120416 -0.472176 -0.462981   
..           ...       ...       ...   ...  ...       ...       ...       ...   
867     1.649312       0.0 -1.585850   697    2  0.110020 -0.472176 -0.462981   
193    -0.986860       1.0 -0.379115   580    2 -2.040715  0.468220  0.811558   
14     -1.686971       0.0  0.827621   841    1 -1.195784 -0.472176 -0.462981   
150    -1.155043       0.0 -0.379115    69    2  1.646259 -0.472176 -0.462981   
580     0.526788       1.0 -0.379115   161    1 -0.350852  0.468220  0.811558   

     Ticket      Fare  ... 

using 'xs' to get the table in the manner we are used to see it. Notice that it is without 'Survived' column

In [ ]:
#collapse-show
to.train.xs

,Name,Sex,Ticket,Cabin,Embarked,Name_wiki,Hometown,Boarded,Destination,Lifeboat,...,Class_na,PassengerId,Pclass,SibSp,Parch,Age,Fare,WikiId,Age_wiki,Class
430,93,2,4,72,3,89,354,4,223,22,...,1,-0.059897,-1.585850,-0.472176,-0.462981,-0.120416,-0.076707,-1.694354,-0.099986,-1.585853
158,768,2,259,0,3,732,300,4,148,0,...,1,-1.123753,0.827621,-0.472176,-0.462981,-0.120416,-0.525029,1.479525,0.551559,0.827622
453,291,2,85,87,1,272,291,2,149,13,...,1,0.030061,-1.585850,0.468220,-0.462981,1.492635,1.491114,-1.430308,1.275499,-1.585853
750,853,1,235,0,3,818,172,4,4,6,...,1,1.191698,-0.379115,0.468220,0.811558,-1.963903,-0.165683,-0.195430,-1.837440,-0.379115
250,674,2,423,0,3,642,293,4,148,0,...,1,-0.763919,0.827621,-0.472176,-0.462981,-0.120416,-0.560431,1.268822,-0.751531,0.827622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,697,2,591,7,3,663,385,4,216,0,...,1,1.649312,-1.585850,-0.472176,-0.462981,0.110020,0.523456,-1.147593,0.117196,-1.585853
193,580,2,114,142,3,555,280,4,148,22,...,1,-0.986860,-0.379115,0.468220,0.811558,-2.040715,-0.090492,-0.387463,-1.909834,-0.379115
14,841,1,414,0,3,806,324,4,123,0,...,1,-1.686971,0.827621,-0.472176,-0.462981,-1.195784,-0.545288,1.650221,-1.113501,0.827622
150,69,2,623,0,3,69,194,4,107,0,...,1,-1.155043,-0.379115,-0.472176,-0.462981,1.646259,-0.428222,-0.878214,1.565074,-0.379115


using the same for target value

In [ ]:
#collapse-show
to.train.ys.values.ravel()

array([1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 1.

# Model:

Now, we got our data preprocessed. It's time to model the data. For this problem i am using RandomForestClassifier it's powerful and simple. And taking out X_train and y_train values like we showed before examples.

In [ ]:
### RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_valid, y_valid = to.valid.xs, to.valid.ys.values.ravel()

We have table without any hardcore preprocessing all we did was just to use fastai tabular function to get this

In [ ]:
X_train.head()

,Name,Sex,Ticket,Cabin,Embarked,Name_wiki,Hometown,Boarded,Destination,Lifeboat,...,Class_na,PassengerId,Pclass,SibSp,Parch,Age,Fare,WikiId,Age_wiki,Class
430,93,2,4,72,3,89,354,4,223,22,...,1,-0.059897,-1.585850,-0.472176,-0.462981,-0.120416,-0.076707,-1.694354,-0.099986,-1.585853
158,768,2,259,0,3,732,300,4,148,0,...,1,-1.123753,0.827621,-0.472176,-0.462981,-0.120416,-0.525029,1.479525,0.551559,0.827622
453,291,2,85,87,1,272,291,2,149,13,...,1,0.030061,-1.585850,0.468220,-0.462981,1.492635,1.491114,-1.430308,1.275499,-1.585853
750,853,1,235,0,3,818,172,4,4,6,...,1,1.191698,-0.379115,0.468220,0.811558,-1.963903,-0.165683,-0.195430,-1.837440,-0.379115
250,674,2,423,0,3,642,293,4,148,0,...,1,-0.763919,0.827621,-0.472176,-0.462981,-0.120416,-0.560431,1.268822,-0.751531,0.827622


Model initialization using sklearn. We are simply using 100 different decision trees. Keeping the model parameters simple as possible so that we don't overfit either.

In [ ]:
rnf_classifier= RandomForestClassifier(n_estimators=100, n_jobs=-1)
rnf_classifier.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1)

We just Trained randomforest classifier and predicting accuracy on validation set

## Accuracy on validation dataset

In [ ]:
y_pred=rnf_classifier.predict(X_valid)


from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_valid)

0.9943820224719101

WOW, we get the accuracy of 0.99 on validation dataset this means on leader board getting rank of 200 something our of 19,000. 

# TEST dataset

We need to turn the test data format as intended so that to out feed in model. Taking look at test data

In [ ]:
#hide_input
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,WikiId,Name_wiki,Age_wiki,Hometown,Boarded,Destination,Lifeboat,Body,Class
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,928.0,"Kelly, Mr. James",19.0,"Unknown, Ireland",Southampton,New York City,NaN,70MB,3.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1297.0,"Wilkes, Mrs. Ellen",47.0,"Penzance, Cornwall, England",Southampton,"Akron, Ohio, US",16,NaN,3.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,518.0,"Myles, Mr. Thomas Francis",63.0,"Fermoy, Ireland[note 1]",Queenstown,"Waban, Massachusetts, US",NaN,NaN,2.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1303.0,"Wirz, Mr. Albert",27.0,"Uster, Switzerland",Southampton,"Beloit, Wisconsin, US",NaN,131MB,3.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,871.0,"Hirvonen, Mrs. Helga Elisabeth (née Lindqvist)",22.0,"Taalintehdas, Finland",Southampton,"Monessen, Pennsylvania, US",15,NaN,3.0


doing the same preprocessing we did for training set

In [ ]:
df_test.dtypes
g_train =df_test.columns.to_series().groupby(df_test.dtypes).groups
g_train

{int64: ['PassengerId', 'Pclass', 'SibSp', 'Parch'], float64: ['Age', 'Fare', 'WikiId', 'Age_wiki', 'Class'], object: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Name_wiki', 'Hometown', 'Boarded', 'Destination', 'Lifeboat', 'Body']}

In [ ]:
#collapse-show
cat_names= [
        'Name', 'Sex', 'Ticket', 'Cabin', 
        'Embarked', 'Name_wiki', 'Hometown', 
        'Boarded', 'Destination', 'Lifeboat', 
        'Body'
]

cont_names = [ 
    'PassengerId', 'Pclass', 'SibSp', 'Parch', 
    'Age', 'Fare', 'WikiId', 'Age_wiki','Class'
 ]

Now further, Avoiding y_name and split for obvious reasons. 

In [ ]:
#collapse-show
test = TabularPandas(df_test, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   )

getting the table value for test dataset. we used train.xs becoz it lets us extact the data we had given the tabularpandas function


In [ ]:
#hide
X_test= test.train.xs

In [ ]:
#collapse-show
X_test.head()

,Name,Sex,Ticket,Cabin,Embarked,Name_wiki,Hometown,Boarded,Destination,Lifeboat,...,Class_na,PassengerId,Pclass,SibSp,Parch,Age,Fare,WikiId,Age_wiki,Class
0,207,2,153,0,2,194,226,4,103,0,...,1,-1.727912,0.873482,-0.499470,-0.400248,0.386231,-0.497413,0.752828,-0.794545,0.871140
1,404,1,222,0,3,389,162,4,2,9,...,1,-1.719625,0.873482,0.616992,-0.400248,1.371369,-0.512278,1.729823,1.304755,0.871140
2,270,2,74,0,2,257,65,3,154,0,...,1,-1.711337,-0.315819,-0.499470,-0.400248,2.553536,-0.464100,-0.332721,2.504355,-0.311121
3,409,2,148,0,3,393,233,4,13,0,...,1,-1.703050,0.873482,-0.499470,-0.400248,-0.204852,-0.482475,1.745709,-0.194745,0.871140
4,179,1,139,0,3,167,208,4,94,7,...,1,-1.694763,0.873482,0.616992,0.619896,-0.598908,-0.417491,0.601910,-0.569620,0.871140


Now taking a look at features present in test data.

In [ ]:
#collapse-show
X_test.dtypes
g_train =X_test.columns.to_series().groupby(X_test.dtypes).groups
g_train

{int8: ['Sex', 'Cabin', 'Embarked', 'Boarded', 'Lifeboat', 'Body', 'Age_na', 'Fare_na', 'WikiId_na', 'Age_wiki_na', 'Class_na'], int16: ['Name', 'Ticket', 'Name_wiki', 'Hometown', 'Destination'], float64: ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare', 'WikiId', 'Age_wiki', 'Class']}

 > Warning: See that we are dropping `Fare_na` column

In [ ]:
X_test= X_test.drop('Fare_na', axis=1)

Now you would be thinking why we are dropping the 'Fare_na' column. The reason is, originally we trained model on 24 features. But as you know that tabularpandas function creates a new column if there is any missing value spotted in any particular feature.

Here, our test data had some Fare values missing so the function created the boolean column this lead to 25 features. If you try to pass without dropping Fare_na column, it will give you the error that model was trained on 24 features and we are passing 25. That's why dropping Fare_na feature. And dont worry it doesnt have any effect on the output

In [ ]:
#hide
y_pred=rnf_classifier.predict(X_test)


In [ ]:
#hide
y_pred= y_pred.astype(int)

Here is our output

In [ ]:
output= pd.DataFrame({'PassengerId':df_test.PassengerId, 'Survived': y_pred})
output.to_csv('my_submission_titanic.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
